In [54]:
import hyper_param as hp
import evaluate
import numpy as np
import operator
import collections
import random
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = 20
#logging.info("")

('average:', array([-7.35566667]), 'standard_error', 18.455453223347735)
uniform_baseline
('average:', array([ 79.31]), 'standard_error', 147.20489170829859)


In [37]:
#Define constants
tau = 10 #price history window
t_0 = 1000 #start point in history
num_dyna_updates = 25
ntrials = 10000
epsilon = 0.2
horizon = 100
gamma = 0.9

In [64]:
price = hp.get_price_history()

In [65]:
len(price)

2593

In [66]:
'''
Returns r, sp
'''
def sim(t, s, a):
    x = [t+i for i in range(tau)]
    y = price[t-tau:t]
    slope, intercept = np.polyfit(x, y, 1)
    r = 0
    ncoin = s[1]
    if ncoin <= -50 or ncoin >= 50:
        return ((-price[t]*ncoin)[0], (hp.get_interval_enum(slope[0]), 0))
    else:
        if a == -1: # buy
            r = 0-price[t][0]
            ncoin += 1
        elif a == 1: # sell
            r = price[t][0]
            ncoin -= 1
        elif a == 0:
            reward = 0
        else:
            #throw expection
            print("Invalid action")
        sp = (hp.get_interval_enum(slope[0]), ncoin)
        return r, sp

In [67]:
'''
initialize N's, p, Q
'''
N_sasp = {} # key = N_sasp[(s, a)][sp] #value = transition count
N_sa = collections.defaultdict(int) #key = (s, a) #value = occurence count
p = collections.defaultdict(int) #key = (s, a) #value = sum of rewards
Q = {} # Q[s][a] = q value of taking action a in state s

In [68]:
'''
Returns chosen action
'''
def choose_action_egreedy(s, Q):
    if s in Q:
        Q_value = Q[s]
        prob = random.uniform(0, 1)
        if prob <= 1 - epsilon:  #exploit
            action = max(Q_value, key=Q_value.get)
            return action
        else:  #explore
            action = random.choice([-1, 0, 1])
            if action in Q_value:
                return action
            else:
                return action
    else:
        action = random.choice([-1, 0, 1])
        return action

In [69]:
def get_new_random_state_action(Q, updated):
    for i in range(5):
        rand_s = random.choice(Q.keys())
        rand_a = random.choice(Q[rand_s].keys())
        if (rand_s, rand_a) not in updated:
            return rand_s, rand_a
    return None, None

In [70]:
def dyna(Q, s, a, N_sa, N_sasp, p):
    R_sa = p[(s, a)]/ N_sa[(s, a)]
    
    summation = 0
    for sp in N_sasp[(s, a)]:
        T_sasp = N_sasp[(s, a)][sp]/N_sa[(s, a)]
        #compute max_a'[Q(s',a')]
        if sp not in Q:
            continue #Q = 0
        max_Q = max(Q[sp].iteritems(), key=operator.itemgetter(1))[1]
        summation += T_sasp*max_Q
    
    if s not in Q:
        Q[s] = {}
    if a not in Q[s]:
        Q[s][a] = 0
        
    Q[s][a] = R_sa + gamma*summation

In [72]:
def mle_based_rl(t, s_0):
    s = s_0   #initial state (ncoins, slope_interval)
    curr_horizon = t + horizon
    while (t < curr_horizon):
        
        a = choose_action_egreedy(s, Q)
        r, sp = sim(t, s, a)

        #update N counts
        if (s , a) not in N_sa:
            N_sasp[(s, a)] = collections.defaultdict(int)
            
        N_sa[(s, a)] += 1
        N_sasp[(s, a)][sp] += 1
        
        #update p
        p[(s, a)] += r
        
        dyna(Q, s, a, N_sa, N_sasp, p)
        
        #update Q using Dyna strategy
        updated = set((s,a))
        for i in range(num_dyna_updates):
            s_rand, a_rand = get_new_random_state_action(Q, updated)
            if s_rand is None:   # nothing to update
                break
            dyna(Q, s_rand, a_rand, N_sa, N_sasp, p)
            updated.add((s_rand, a_rand))
        
        s = sp
        t += 1

In [77]:
for i in range(ntrials):
    t = random.randint(t_0, len(price)-100)
    s_0 = (0, 0)
    if t > t_0:
        x = [t+i for i in range(tau)]
        y = price[t-tau:t]
        slope, intercept = np.polyfit(x, y, 1)
        s_0 = (hp.get_interval_enum(slope[0]), 0)
    
    #logging.info("MLE(t=%d, s_0=[%d, %d])", t, s_0[0], s_0[1])
    mle_based_rl(t, s_0)

In [81]:
print(Q)

with open('./mle_log.txt', 'w') as f:
     f.write(str(Q))


{(12, -43): {0: 0.0, 1: 19.070745192307676, -1: -23.941206896551723}, (7, 3): {-1: -36.524000000000001}, (18, -29): {0: 0.0, 1: 60.375413793103455, -1: -60.806296296296303}, (11, -34): {0: 0.0, 1: 5.7404822938688769, -1: -5.1631239669421465}, (7, -37): {0: 0.0, 1: 66.82943060498215, -1: -57.274516129032264}, (17, -32): {0: 0.0, 1: 63.370300751879654, -1: -55.703043478260859}, (10, -45): {0: 0.0, 1: 6.783327664399101, -1: -6.3805996472663216}, (4, -50): {0: 3165.7272727272725, 1: 2482.1034482758619, -1: 3084.8888888888887}, (4, 4): {-1: -47.01699}, (6, -34): {0: 0.0, 1: 63.903623188405852, -1: -54.599999999999994}, (16, -31): {0: 0.0, 1: 39.671240310077565, -1: -36.701785714285727}, (18, -3): {0: 0.0, 1: 59.006686930091199, -1: -56.328387096774186}, (17, -45): {0: 0.0, 1: 64.289413265306095, -1: -55.964705882352931}, (20, -23): {0: 0.0, 1: 129.84529608259973, -1: -124.45566787003618}, (5, -43): {0: 0.0, 1: 94.855895953757269, -1: -111.24923076923076}, (7, -15): {0: 0.0, 1: 64.9085146804

In [82]:
def generate_mle_policy(Q, time, price):
    policy = []
    cur_num_coins = 0
    for t in range(time, time+hp.get_policy_length()):
        logging.info(t)
        x = [t+i for i in range(tau)]
        logging.info(x)
        y = price[t-tau:t]
        logging.info(y)
        slope, intercept = np.polyfit(x, y, 1)
        interval_group = hp.get_interval_enum(slope[0])
        if (interval_group, cur_num_coins) in Q: 
            Q_value = Q[(interval_group, cur_num_coins)]
            action = max(Q_value, key=Q_value.get)
            if Q_value[action] < 0: # meaningless maximum
                action = random.choice([-1, 0, 1])
        else:
            action = random.choice([-1, 0, 1])
            
        if action == -1:
            cur_num_coins += 1
        elif action == 1:
            cur_num_coins -= 1
        else:
            pass
        
        policy.append(action)
    return policy

In [83]:
reward_array = []
for i in range(hp.get_evaluation_step()):
    random_time = random.randint(1000, 2000)
    policy = generate_mle_policy(Q, random_time, price)
    reward = evaluate.evaluate_reward(policy, price, random_time)
    reward_array.append(reward)

hp.print_statistics_95_confidence(reward_array)

INFO : 1235
INFO : [1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244]
INFO : [[ 1.82]
 [ 1.94]
 [ 1.84]
 [ 1.79]
 [ 1.86]
 [ 1.83]
 [ 1.84]
 [ 1.85]
 [ 1.85]
 [ 1.8 ]]
INFO : 1236
INFO : [1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245]
INFO : [[ 1.94]
 [ 1.84]
 [ 1.79]
 [ 1.86]
 [ 1.83]
 [ 1.84]
 [ 1.85]
 [ 1.85]
 [ 1.8 ]
 [ 1.81]]
INFO : 1237
INFO : [1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246]
INFO : [[ 1.84]
 [ 1.79]
 [ 1.86]
 [ 1.83]
 [ 1.84]
 [ 1.85]
 [ 1.85]
 [ 1.8 ]
 [ 1.81]
 [ 1.83]]
INFO : 1238
INFO : [1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247]
INFO : [[ 1.79]
 [ 1.86]
 [ 1.83]
 [ 1.84]
 [ 1.85]
 [ 1.85]
 [ 1.8 ]
 [ 1.81]
 [ 1.83]
 [ 1.81]]
INFO : 1239
INFO : [1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248]
INFO : [[ 1.86]
 [ 1.83]
 [ 1.84]
 [ 1.85]
 [ 1.85]
 [ 1.8 ]
 [ 1.81]
 [ 1.83]
 [ 1.81]
 [ 1.8 ]]
INFO : 1240
INFO : [1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249]
INFO : [[ 1.83]
 [ 1.84]
 [ 1.

INFO : [1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290]
INFO : [[ 1.65]
 [ 1.67]
 [ 1.68]
 [ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]
 [ 1.59]]
INFO : 1282
INFO : [1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291]
INFO : [[ 1.67]
 [ 1.68]
 [ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]
 [ 1.59]
 [ 1.52]]
INFO : 1283
INFO : [1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292]
INFO : [[ 1.68]
 [ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]
 [ 1.59]
 [ 1.52]
 [ 1.46]]
INFO : 1284
INFO : [1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293]
INFO : [[ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]
 [ 1.59]
 [ 1.52]
 [ 1.46]
 [ 1.47]]
INFO : 1285
INFO : [1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294]
INFO : [[ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]
 [ 1.59]
 [ 1.52]
 [ 1.46]
 [ 1.47]
 [ 1.38]]
INFO : 1286
INFO : [1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295]
INFO : [[ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]

INFO : [[ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]]
INFO : 1328
INFO : [1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337]
INFO : [[ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]
 [ 1.82]]
INFO : 1329
INFO : [1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338]
INFO : [[ 1.45]
 [ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]
 [ 1.82]
 [ 1.85]]
INFO : 1330
INFO : [1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339]
INFO : [[ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]
 [ 1.82]
 [ 1.85]
 [ 1.84]]
INFO : 1331
INFO : [1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340]
INFO : [[ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]
 [ 1.82]
 [ 1.85]
 [ 1.84]
 [ 1.83]]
INFO : 1332
INFO : [1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341]
INFO : [[ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]
 [ 1.82]
 [ 1.85]
 [ 1.84]
 [ 1.83]
 [ 1.81]]
INFO : 1333


INFO : 1922
INFO : [1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931]
INFO : [[ 4.33]
 [ 4.52]
 [ 4.66]
 [ 4.63]
 [ 4.91]
 [ 4.3 ]
 [ 3.86]
 [ 3.96]
 [ 4.  ]
 [ 4.3 ]]
INFO : 1923
INFO : [1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932]
INFO : [[ 4.52]
 [ 4.66]
 [ 4.63]
 [ 4.91]
 [ 4.3 ]
 [ 3.86]
 [ 3.96]
 [ 4.  ]
 [ 4.3 ]
 [ 4.6 ]]
INFO : 1924
INFO : [1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933]
INFO : [[ 4.66]
 [ 4.63]
 [ 4.91]
 [ 4.3 ]
 [ 3.86]
 [ 3.96]
 [ 4.  ]
 [ 4.3 ]
 [ 4.6 ]
 [ 3.84]]
INFO : 1925
INFO : [1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934]
INFO : [[ 4.63]
 [ 4.91]
 [ 4.3 ]
 [ 3.86]
 [ 3.96]
 [ 4.  ]
 [ 4.3 ]
 [ 4.6 ]
 [ 3.84]
 [ 3.95]]
INFO : 1926
INFO : [1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935]
INFO : [[ 4.91]
 [ 4.3 ]
 [ 3.86]
 [ 3.96]
 [ 4.  ]
 [ 4.3 ]
 [ 4.6 ]
 [ 3.84]
 [ 3.95]
 [ 3.89]]
INFO : 1927
INFO : [1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936]
INFO : [[ 4.3 ]
 [ 3.86]
 [ 3.

INFO : [1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977]
INFO : [[ 3.8 ]
 [ 3.78]
 [ 3.83]
 [ 3.79]
 [ 3.77]
 [ 3.76]
 [ 3.79]
 [ 3.8 ]
 [ 3.83]
 [ 3.85]]
INFO : 1969
INFO : [1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978]
INFO : [[ 3.78]
 [ 3.83]
 [ 3.79]
 [ 3.77]
 [ 3.76]
 [ 3.79]
 [ 3.8 ]
 [ 3.83]
 [ 3.85]
 [ 3.82]]
INFO : 1970
INFO : [1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979]
INFO : [[ 3.83]
 [ 3.79]
 [ 3.77]
 [ 3.76]
 [ 3.79]
 [ 3.8 ]
 [ 3.83]
 [ 3.85]
 [ 3.82]
 [ 3.81]]
INFO : 1971
INFO : [1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980]
INFO : [[ 3.79]
 [ 3.77]
 [ 3.76]
 [ 3.79]
 [ 3.8 ]
 [ 3.83]
 [ 3.85]
 [ 3.82]
 [ 3.81]
 [ 3.8 ]]
INFO : 1972
INFO : [1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981]
INFO : [[ 3.77]
 [ 3.76]
 [ 3.79]
 [ 3.8 ]
 [ 3.83]
 [ 3.85]
 [ 3.82]
 [ 3.81]
 [ 3.8 ]
 [ 3.77]]
INFO : 1973
INFO : [1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982]
INFO : [[ 3.76]
 [ 3.79]
 [ 3.8 ]
 [ 3.83]

...

INFO : [1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288]
INFO : [[ 1.66]
 [ 1.67]
 [ 1.65]
 [ 1.67]
 [ 1.68]
 [ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]]
INFO : 1280
INFO : [1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289]
INFO : [[ 1.67]
 [ 1.65]
 [ 1.67]
 [ 1.68]
 [ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]]
INFO : 1281
INFO : [1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290]
INFO : [[ 1.65]
 [ 1.67]
 [ 1.68]
 [ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]
 [ 1.59]]
INFO : 1282
INFO : [1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291]
INFO : [[ 1.67]
 [ 1.68]
 [ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]
 [ 1.59]
 [ 1.52]]
INFO : 1283
INFO : [1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292]
INFO : [[ 1.68]
 [ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]
 [ 1.67]
 [ 1.62]
 [ 1.59]
 [ 1.52]
 [ 1.46]]
INFO : 1284
INFO : [1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293]
INFO : [[ 1.7 ]
 [ 1.68]
 [ 1.7 ]
 [ 1.69]

INFO : [[ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]]
INFO : 1326
INFO : [1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335]
INFO : [[ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]]
INFO : 1327
INFO : [1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336]
INFO : [[ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]]
INFO : 1328
INFO : [1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337]
INFO : [[ 1.45]
 [ 1.45]
 [ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]
 [ 1.82]]
INFO : 1329
INFO : [1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338]
INFO : [[ 1.45]
 [ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]
 [ 1.82]
 [ 1.85]]
INFO : 1330
INFO : [1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339]
INFO : [[ 1.45]
 [ 1.46]
 [ 1.48]
 [ 1.83]
 [ 1.79]
 [ 1.83]
 [ 1.8 ]
 [ 1.82]
 [ 1.85]
 [ 1.84]]
INFO : 1331


INFO : 1290
INFO : [1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299]
INFO : [[ 1.59]
 [ 1.52]
 [ 1.46]
 [ 1.47]
 [ 1.38]
 [ 1.38]
 [ 1.38]
 [ 1.42]
 [ 1.41]
 [ 1.41]]
INFO : 1291
INFO : [1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300]
INFO : [[ 1.52]
 [ 1.46]
 [ 1.47]
 [ 1.38]
 [ 1.38]
 [ 1.38]
 [ 1.42]
 [ 1.41]
 [ 1.41]
 [ 1.38]]
INFO : 1292
INFO : [1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301]
INFO : [[ 1.46]
 [ 1.47]
 [ 1.38]
 [ 1.38]
 [ 1.38]
 [ 1.42]
 [ 1.41]
 [ 1.41]
 [ 1.38]
 [ 1.41]]
INFO : 1293
INFO : [1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302]
INFO : [[ 1.47]
 [ 1.38]
 [ 1.38]
 [ 1.38]
 [ 1.42]
 [ 1.41]
 [ 1.41]
 [ 1.38]
 [ 1.41]
 [ 1.43]]
INFO : 1294
INFO : [1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303]
INFO : [[ 1.38]
 [ 1.38]
 [ 1.38]
 [ 1.42]
 [ 1.41]
 [ 1.41]
 [ 1.38]
 [ 1.41]
 [ 1.43]
 [ 1.46]]
INFO : 1295
INFO : [1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304]
INFO : [[ 1.38]
 [ 1.38]
 [ 1.

INFO : [1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345]
INFO : [[ 1.8 ]
 [ 1.82]
 [ 1.85]
 [ 1.84]
 [ 1.83]
 [ 1.81]
 [ 1.64]
 [ 1.62]
 [ 1.67]
 [ 1.69]]
INFO : 1337
INFO : [1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346]
INFO : [[ 1.82]
 [ 1.85]
 [ 1.84]
 [ 1.83]
 [ 1.81]
 [ 1.64]
 [ 1.62]
 [ 1.67]
 [ 1.69]
 [ 1.67]]
INFO : 1338
INFO : [1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347]
INFO : [[ 1.85]
 [ 1.84]
 [ 1.83]
 [ 1.81]
 [ 1.64]
 [ 1.62]
 [ 1.67]
 [ 1.69]
 [ 1.67]
 [ 1.76]]
INFO : 1339
INFO : [1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348]
INFO : [[ 1.84]
 [ 1.83]
 [ 1.81]
 [ 1.64]
 [ 1.62]
 [ 1.67]
 [ 1.69]
 [ 1.67]
 [ 1.76]
 [ 1.76]]
INFO : 1340
INFO : [1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349]
INFO : [[ 1.83]
 [ 1.81]
 [ 1.64]
 [ 1.62]
 [ 1.67]
 [ 1.69]
 [ 1.67]
 [ 1.76]
 [ 1.76]
 [ 1.73]]
INFO : 1341
INFO : [1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350]
INFO : [[ 1.81]
 [ 1.64]
 [ 1.62]
 [ 1.67]

AttributeError: 'module' object has no attribute 'print_statistics_95_confidence'